In [1]:
import numpy as np
import imageio

# Thomas' cyclically symmetric attractor

http://rreusser.github.io/strange-attractors/#thomas  
https://en.wikipedia.org/wiki/Thomas%27_cyclically_symmetric_attractor

In [2]:
def step_RK4(dt, t, Y, dYdt):
    ''' Runge Kutta 4th order integrator
        
        Y:  nd array of shape (number of dim, numbers of points)
    '''
   
    t_plus_dt = t + dt
    k1 = dYdt(t, Y)
    k2 = dYdt(t_plus_dt, Y + dt/2*k1)
    k3 = dYdt(t_plus_dt, Y + dt/2*k2)
    k4 = dYdt(t_plus_dt, Y + dt*k3)

    Ynext = Y + dt/6*(k1 + 2*k2 + 2*k3 + k4)
    
    return t_plus_dt, Ynext

In [81]:
def thomas_attractor(xyz, b=0.19):
    # xyz shape (dim, nbr points)
    sin_xyz = np.sin(xyz)
    dYdt = -b*np.copy(xyz)
    dYdt[0, :] += sin_xyz[1, :]
    dYdt[1, :] += sin_xyz[2, :]
    dYdt[2, :] += sin_xyz[0, :]
    return dYdt


## Fill the cube

In [181]:
b = 0.19
n_points = 10000

Y = np.random.rand(3, n_points)*12 - 6 

t, dt = 0, 0.1

def dYdt(t, Y):
    return thomas_attractor(Y, b=b)

# burn - to reach stable orbits
for _ in range(3000):
    t, Y = step_RK4(0.1, t, Y, dYdt)

In [218]:
half_attractor_size = 4.2  # demi axe
nbr_pixel = 700

cube = np.zeros((nbr_pixel, nbr_pixel, nbr_pixel))  # huge in memory...

print("%f Gbytes" % (cube.size * cube.itemsize*1e-9))

2.744000 Gbytes


In [231]:
# accumulate pixel value on a cube
t, dt = 0, 0.005

store_Y = []
for k in range(5000):
    dt = 0.01 + 0.001*np.random.rand()
    t, Y = step_RK4(dt, t, Y, dYdt)
    Y_px = ( (Y + half_attractor_size)/(2*half_attractor_size) * nbr_pixel ).astype(int) #np.rint
    #z_scale = (Y_px[2, :]/nbr_pixel)**2  # shade function of Z
    value = cube[Y_px[0, :], Y_px[1, :], Y_px[2, :]]
    np.add.at(cube, (Y_px[0, :], Y_px[1, :], Y_px[2, :]), 1)#1/(1+value))
    # see https://stackoverflow.com/a/28894452/8069403
    
    if k%200 == 0:
        print('.', end='')
        
print('\n done')

.........................
 done


In [232]:
# Sort of Z buffer
count_threshold = 1
nz = np.nonzero( np.max(cube, axis=2) > count_threshold )
image = np.zeros((nbr_pixel, nbr_pixel))
visible_voxel = []
for i, j in zip(*nz):
    col = cube[i, j, :]
    idx_max = np.max(np.nonzero(col > count_threshold))
    image[i, j] = col[idx_max]
    visible_voxel.append((i, j, idx_max))

In [233]:
# Save as image
image = image / image.max()
image[image<0.0001] = 0.3  # set background
image = 255 * (1 - image)**2

image = image.astype(np.uint8)

imageio.imwrite('from_cube.png', image)

![cube](from_cube.png)

- Is a point on the manifold? (is it a actually a manifold?)
    - burn steps
    - backward integration?
    
- go 3d 
    - https://fr.wikipedia.org/wiki/Octree
    - marching cube render
    - three js with lines
    - potree
    
    
- memory --> use sparse cube array
- density & velocity?  what is density ?
    - mesure de la divergence d'un point -statistique  
    https://fr.wikipedia.org/wiki/Exposant_de_Liapounov

In [151]:
len(visible_voxel)

177943

In [156]:
divergence_map = np.zeros((nbr_pixel, nbr_pixel))
for i, j, z in visible_voxel:
    initial_std = 0.1
    n_points = 20
    dt = 0.1
    xyz = (np.array([i, j, z])/nbr_pixel - 0.5)*half_attractor_size*2
    Y = np.random.randn(3, n_points)*initial_std + xyz[:, np.newaxis]
    for _ in range(15):
        t, Y = step_RK4(dt, t, Y, dYdt)
     
    final_std = np.std(Y, axis=1).mean()
    divergence_map[i, j] = np.log(final_std/initial_std)
    

In [157]:
# Save as image
image = np.copy(divergence_map)
image = image / image.max()
image[image<0.0001] = 0.3  # set background
image = 255 * (1 - image)

image = image.astype(np.uint8)

imageio.imwrite('divergence_log.png', image)

### Animation without matplotlib

In [251]:
b = 0.19
n_points = 1500

Y = np.random.rand(3, n_points)*12 - 6 

t, dt = 0, 0.01

def dYdt(t, Y):
    return thomas_attractor(Y, b=b)

# burn
for _ in range(5000):
    t, Y = step_RK4(dt, t, Y, dYdt)

In [252]:
half_attractor_size = 4.2  # demi axe
nbr_pixel = 650

image = np.zeros((nbr_pixel, nbr_pixel))

In [253]:
n_fig = 150

for k in range(n_fig):
    image = np.zeros((nbr_pixel, nbr_pixel))
    for i in range(100):
        t, Y = step_RK4(dt, t, Y, dYdt)
        
        Y_px = np.rint( (Y + half_attractor_size)/(2*half_attractor_size) * nbr_pixel ).astype(int)
        #np.add.at(image, (Y_px[0, :], Y_px[1, :]), z_scale)
        z_scale = 0.6*(Y_px[2, :]/nbr_pixel)**2 + 0.4 # shade function of Z
        image[Y_px[0, :], Y_px[1, :]] = z_scale

        # starting point for the next frame
        if i == 25:
            Y_next = np.copy(Y)
    
    Y = Y_next
    
    image = image / image.max()
    image = 255 * (1 - image)
    image = image.astype(np.uint8)

    imageio.imwrite(f'anim/image_{k:04d}.png', image)
    print(k, end=' ')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 

ImageMagick 
    
    convert -delay 10   -loop 0   image_*.png  animated_attractor.gif
